# Chapter 5 - Django Forms

## Django Form Structure and Workflow

In [ ]:
from django import forms
from django.shortcuts import render
from django.http import HttpResponseRedirect


6.1. Django form class definition

In [ ]:
# forms.py in 'contact' app 

class ContactForm(forms.Form):
    name = forms.CharField(required=False)
    email = forms.EmailField(label='Your email')
    comment = forms.CharField(widget=forms.Textarea)


6.2. Django view method that uses a Django form

In [ ]:
# views.py in 'contact' app
from .forms import ContactForm


def contact(request):
    form = ContactForm()
    return render(request,'about/contact.html', {'form':form})


6.3. Django form instance rendered in template as HTML

In [ ]:
<tr>
    <th><label for="id_name">Name:</label></th>
    <td><input id="id_name" name="name" type="text"/></td>
</tr>
<tr>
    <th><label for="id_email">Your email:</label></th>
    <td><input id="id_email" required name="email" type="email"/></td>
</tr>
<tr>
    <th><label for="id_comment">Comment:</label></th>
    <td>
        <textarea cols="40" id="id_comment" required name="comment" rows="10"></textarea>
    </td>
</tr>


## Functional Web Form Syntax for Django Forms

6.4. Django form template declaration for functional web form

In [ ]:
<form method="POST">
    {% csrf_token %}
    <table>
        {{form.as_table}}
    </table>
    <input type="submit" value="Submit form">
</form>


## Django View Method to Process Form (POST Handling)

In [ ]:
from django.views.decorators.csrf import csrf_exempt, csrf_protect


6.5. Django view method that sends and processes Django form


In [ ]:
def contact(request):
    if request.method == 'POST':
        # POST, generate form with data from the request
        form = ContactForm(request.POST)
        # check if it's valid:
        if form.is_valid():
            # process data, insert into DB, generate email,etc
            # redirect to a new URL:
            return HttpResponseRedirect('/about/contact/thankyou')
    else:
        # GET, generate blank form
        form = ContactForm()
    return render(request,'about/contact.html', {'form':form})


6.6. Django view method decorated with @csrf_exempt() to bypass CSRF enforcement


In [ ]:
@csrf_exempt
def contact(request):
    # Any POST-processing inside view method
    # ignores if there is or isn't a CSRF token
    return

6.7. Django view method decorated with @csrf_protect() to enforce CSRF when CSRF is disabled at 
the project level

In [ ]:
@csrf_protect
def contact(request):
    # Any POST processing inside view method
    # checks for the presence of a CSRF token
    # even when CsrfViewMiddleware is removed
    return


## Django Form Processing: Initialization, Field Access, Validation, and Error Handling

In [ ]:
import re
from django import forms
from django.shortcuts import render
from django.http import HttpResponseRedirect


6.8. Django form class with backing processing view method

In [ ]:
class ContactForm(forms.Form):
    name = forms.CharField(required=False)
    email = forms.EmailField(label='Your email')
    comment = forms.CharField(widget=forms.Textarea)


def contact(request):
    if request.method == 'POST':
        form = ContactForm(request.POST)
    else:
        form = ContactForm()
    return render(request, 'about/contact.html', {'form':form})


6.9. Django form instance with initial argument declared in view method

In [ ]:
def contact(request):
    if request.method == 'POST':
        #...
        #...
        pass
    else:
        # GET,  generate blank form
        form = ContactForm(initial={'email':'johndoe@coffeehouse.com', 'name':'John Doe'})
    return render(request, 'about/contact.html', {'form':form})


6.10. Django form fields with initial argument

In [ ]:
class ContactForm(forms.Form):
    name = forms.CharField(required=False, initial='Please provide your name')
    email = forms.EmailField(label='Your email',  initial='We need your email')
    comment = forms.CharField(widget=forms.Textarea)


6.11. Django form initialized with __init__ method

In [ ]:
class ContactForm(forms.Form):
    name = forms.CharField(required=False)
    email = forms.EmailField(label='Your email')
    comment = forms.CharField(widget=forms.Textarea)

    def __init__(self, *args, **kwargs):
        initial_arguments = kwargs.get('initial', None)
        updated_initial = {}
        if initial_arguments:
            user = initial_arguments.get('user',None)
            if user:
                updated_initial['name'] = getattr(user, 'first_name', None)
                updated_initial['email'] = getattr(user, 'email', None)
        updated_initial['comment'] = 'Please provide a comment'
        kwargs.update(initial=updated_initial)
        super(ContactForm, self).__init__(*args, **kwargs)


6.12. Django form with automatic ids (default auto_id=True option) and no automatic ids auto_id=False option


In [ ]:
<!-- Option 1, default auto_id=True -->

<tr>
    <th><label for="id_name">Name:</label></th>
    <td><input id="id_name" name="name" type="text" /></td>
</tr>
<tr>
    <th><label for="id_email">Your email:</label></th>
    <td><input id="id_email" name="email" type="email" /></td>
</tr>
<tr>
    <th><label for="id_comment">Comment:</label></th>
    <td><textarea cols="40" id="id_comment" name="comment" rows="10"></textarea></td>
</tr>

<!-- Option 2 auto_id=False -->

<tr>
    <th>Name:</th><td><input name="name" type="text" /></td>
</tr>
<tr>
    <th>Your email:</th><td><input name="email" type="email" /></td>
</tr>
<tr>
    <th>Comment:</th><td><textarea cols="40" name="comment" rows="10">\r\n</textarea></td>
</tr>


6.13. Django form is_valid() method for form processing

In [ ]:
def contact(request):
    if request.method == 'POST':
        form = ContactForm(request.POST)
        if form.is_valid():
            first_name = form.cleaned_data['first_name']
            email = form.cleaned_data['email']
            return HttpResponseRedirect('/about/contact/thankyou')
        else:
            pass
    else:
        form = ContactForm()
        return render(request, 'about/contact.html', {'form':form})


6.14. Django form field validators option with custom validator method for form processing

In [ ]:
def validate_comment_word_count(value):
    count = len(value.split())
    if count < 30:
        raise forms.ValidationError(
            ('Please provide at least a 30 word message, \%(count)s words is not descriptive enough'), 
            params={'count': count},
        )


class ContactForm(forms.Form):
    name = forms.CharField(required=False)
    email = forms.EmailField(label='Your email')
    comment = forms.CharField(widget=forms.Textarea, validators=[validate_comment_word_count])


6.15. Django form field validation with clean_<field>() methods

In [ ]:
class ContactForm(forms.Form):
    name = forms.CharField(required=False)
    email = forms.EmailField(label='Your email')
    comment = forms.CharField(widget=forms.Textarea)

    def clean_name(self):
        value = self.cleaned_data['name']
        if value.isupper():
            raise forms.ValidationError(
                "Please don't use all upper case for your name, use lower case", 
                code='uppercase'
            )
        return value

    def clean_email(self):
        value = self.cleaned_data['email']
        if value.endswith('@hotmail.com'):
            raise forms.ValidationError(
                "Please don't use a hotmail email, we simply don't like it", 
                code='hotmail'
            )
        return value
    

6.16. Django form field validation with clean() method

In [ ]:
class ContactForm(forms.Form):
    name = forms.CharField(required=False)
    email = forms.EmailField(label='Your email')
    comment = forms.CharField(widget=forms.Textarea)

    def clean(self):
        super(ContactForm, self).clean()
        name = self.cleaned_data.get('name', '')
        email = self.cleaned_data.get('email', '')
        if name.lower() not in email:
            raise forms.ValidationError(
                "Please provide an email that contains your name, or viceversa"
            )


6.17. Django form field error assignment with add_error() in clean() method

In [ ]:
def clean(self):
    super(ContactForm, self).clean()
    name = self.cleaned_data.get('name', '')
    email = self.cleaned_data.get('email', '')
    if name.lower() not in email:
        message = "Please provide an email that contains your name, or viceversa"
        self.add_error('name', message)
        self.add_error('email', forms.ValidationError(message))
        self.add_error(None, message)


6.18. Django form ValidationError instance creation


In [ ]:
# Placed inside def clean_email(self):
raise forms.ValidationError(
    "Please don't use a hotmail email, we simply don't like it",
    code='hotmail'
)

# Placed inside def clean(self):
raise forms.ValidationError([
        forms.ValidationError(
            "Please provide an email that matches your name, or viceversa", 
            code='custom'
        ),
        forms.ValidationError(
            "Please provide your professional email, %(value)s doesn't look professional ", 
            code='required', 
            params={'value': self.cleaned_data.get('email')}
        ),
    ])
